In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

In [2]:
n_hidden = 50
lr = 0.01
epochs = 2000

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)
n_letters = len(char_list)

In [8]:
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list), dtype=int)
    end = np.zeros(shape=len(char_list), dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string :
        idx = char_list.index(i)
        zero = np.zeros(shape=char_len, dtype=int)
        zero[idx] = 1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [9]:
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [15]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
    
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(1, self.hidden_size))
    
rnn = RNN(n_letters, n_hidden, n_letters)

In [18]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [19]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()

    for j in range(one_hot.size()[0]-1):
        input = Variable(one_hot[j:j+1,:])
        output, hidden = rnn.forward(input, hidden)
        target = Variable(one_hot[j+1])
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss
        input = output

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)

tensor(2.2373, grad_fn=<AddBackward0>)
tensor(0.7662, grad_fn=<AddBackward0>)
tensor(0.4222, grad_fn=<AddBackward0>)
tensor(0.2327, grad_fn=<AddBackward0>)
tensor(0.1632, grad_fn=<AddBackward0>)
tensor(0.1173, grad_fn=<AddBackward0>)
tensor(0.0798, grad_fn=<AddBackward0>)
tensor(0.0556, grad_fn=<AddBackward0>)
tensor(0.0486, grad_fn=<AddBackward0>)
tensor(0.0304, grad_fn=<AddBackward0>)
tensor(0.0220, grad_fn=<AddBackward0>)
tensor(0.0222, grad_fn=<AddBackward0>)
tensor(0.0177, grad_fn=<AddBackward0>)
tensor(0.0133, grad_fn=<AddBackward0>)
tensor(0.0108, grad_fn=<AddBackward0>)
tensor(0.0089, grad_fn=<AddBackward0>)
tensor(0.0086, grad_fn=<AddBackward0>)
tensor(0.0102, grad_fn=<AddBackward0>)
tensor(0.0073, grad_fn=<AddBackward0>)
tensor(0.0057, grad_fn=<AddBackward0>)
tensor(0.0051, grad_fn=<AddBackward0>)
tensor(0.0041, grad_fn=<AddBackward0>)
tensor(0.0034, grad_fn=<AddBackward0>)
tensor(0.0028, grad_fn=<AddBackward0>)
tensor(0.0023, grad_fn=<AddBackward0>)
tensor(0.0020, grad_fn=<A

In [20]:
hidden = rnn.init_hidden()
input = Variable(one_hot[0:1, :])

for i in range(len(string)):
    output, hidden = rnn.forward(input, hidden)
    print(onehot_to_word(output.data), end="")
    input = output

hello pytorch. how long can a rnn cell remember?